In [1]:
import pandas as pd

df = pd.read_excel("airline_ticket_dataset.xlsx")
cpi = pd.read_excel("CPI US.xlsx", sheet_name="Monthly")

# Create add_cpi Function
This function is used to convert the monthly CPI into quarterly, then merge it to the Airline Ticket Dataset after adjusting the CPI so that Q1 2022 becomes the base year with CPI = 100.

In [9]:
def add_cpi(airline_ticket: pd.DataFrame, cpi:pd.DataFrame):
    airline_ticket["fare_per_miles"] = airline_ticket["fare"]/df["nsmiles"]
    
    cpi["Year"] = cpi["observation_date"].dt.year
    cpi["month"] = cpi["observation_date"].dt.month
    cpi["quarter"] = (cpi["month"] - 1) // 3 + 1

    cpi_q = (
        cpi.groupby(["Year", "quarter"], as_index=False)
           .agg(cpi_q=("CPIAUCSL", "mean"),
                months_in_q=("CPIAUCSL", "count"))
           .sort_values(["Year", "quarter"])
    )

    cpi_q["cpi_adj"] = cpi_q["cpi_q"]/284.905667 * 100
    cpi_q.drop([14, 15, 16], axis=0, inplace=True)
    
    airline_ticket = airline_ticket.merge(cpi_q[["Year", "quarter", "cpi_adj"]], on=["Year", "quarter"], how="right")
    nom_price = ["fare", "fare_lg", "fare_low"]

    for x in nom_price:
        airline_ticket[x + "_real"] = airline_ticket[x] * (100 / airline_ticket["cpi_adj"])
        
    return airline_ticket

In [10]:
add_cpi(df, cpi)

,Year,quarter,citymarketid_1,citymarketid_2,city1,city2,nsmiles,passengers,fare,carrier_lg,...,TotalPerLFMkts_city1,TotalPerPrem_city1,TotalFaredPax_city2,TotalPerLFMkts_city2,TotalPerPrem_city2,fare_per_miles,cpi_adj,fare_real,fare_lg_real,fare_low_real
0,2022,1,32467,31703,"Miami, FL (Metropolitan Area)","New York City, NY (Metropolitan Area)",1139,17803,179.83,B6,...,0.91560,-0.038550,1.973783e+06,0.848400,-0.055833,0.157884,100.000000,179.830000,165.120000,165.120000
1,2022,1,32575,32457,"Los Angeles, CA (Metropolitan Area)","San Francisco, CA (Metropolitan Area)",372,11272,133.08,WN,...,0.88814,-0.054800,1.969430e+06,0.843000,-0.043600,0.357742,100.000000,133.080000,126.970000,126.970000
2,2022,1,31703,31454,"New York City, NY (Metropolitan Area)","Orlando, FL",989,10113,162.02,B6,...,0.84840,-0.055833,7.232690e+06,0.955100,-0.023300,0.163822,100.000000,162.020000,166.580000,166.580000
3,2022,1,32575,31703,"Los Angeles, CA (Metropolitan Area)","New York City, NY (Metropolitan Area)",2510,9590,315.77,UA,...,0.88814,-0.054800,1.973783e+06,0.848400,-0.055833,0.125805,100.000000,315.770000,305.810000,270.070000
4,2022,1,31703,34027,"New York City, NY (Metropolitan Area)","West Palm Beach/Palm Beach, FL",1056,5952,218.28,B6,...,0.84840,-0.055833,1.440510e+06,0.837000,0.183300,0.206705,100.000000,218.280000,213.730000,213.730000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13999,2025,2,34576,33195,"Rochester, NY","Tampa, FL (Metropolitan Area)",1079,239,184.18,WN,...,0.77910,0.024700,1.609760e+06,0.989733,-0.445167,0.170695,112.593642,163.579396,196.902770,75.066406
14000,2025,2,30994,30198,"Charleston, SC","Pittsburgh, PA",524,239,152.90,MX,...,0.79370,-0.019700,1.810000e+06,0.836300,-0.015600,0.291794,112.593642,135.798076,85.946238,85.946238
14001,2025,2,31714,34492,"Fort Myers, FL","Raleigh/Durham, NC",667,238,145.18,MX,...,0.90050,-0.026500,2.821020e+06,0.908100,-0.050200,0.217661,112.593642,128.941561,76.300934,71.718082
14002,2025,2,33495,34057,"New Orleans, LA","Portland, OR",2050,238,269.35,AS,...,0.96730,-0.050200,3.036620e+06,0.713000,-0.087000,0.131390,112.593642,239.223099,242.642475,225.820922
